In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import os

In [5]:
import json
f = open(os.getcwd()+os.sep+'token.json','r')
access = json.load(f)
f.close()

In [48]:
class idealista_search_api():
  
  def __init__(self,access:dict):
    self.token = access['access_token']
    self.token_type = access['token_type']
    self.api_key = 'p4o5n83p24mfdxhpnzkzzo5rpr2xng1s'
    self.country = 'es'
    self.api_version = '3.5'
    self.endpoint = f'https://api.idealista.com/{self.api_version}/{self.country}/search'
    
  def request(self,payload:dict):
    headers = {
        'Authorization': f'{self.token_type} {self.token}'
    }
    #payload['apikey'] = self.api_key
    response = requests.post(self.endpoint,params=payload,headers=headers)
    if response.status_code == 200:
      return response
    response.raise_for_status()

  def extract_data(self,payload:dict) -> pd.DataFrame:
    payload = {
        'operation':'sale',
        'propertyType':'homes',
        'maxItems':50,
        'order':'price',
        'sort':'desc',
        'locationId':'0-EU-ES-28-07-001-079'
        }
    data = json.loads(self.request(payload).text)
    totalPages = data['totalPages']
    self.dataset = pd.DataFrame(data)

    for page in range(totalPages):
      payload['numPage'] = page
      # append to dataframe



In [56]:
idealista = idealista_search_api(access)
payload = {
    'operation':'sale',
    'propertyType':'homes',
    'maxItems':50,
    'numPage':1,
    'order':'price',
    'sort':'desc',
    'locationId':'0-EU-ES-28-07-001-079'
    #'locationId':'0-EU-ES-28'
    }
response = idealista.request(payload)

In [67]:
type(response)

requests.models.Response

In [57]:
data = json.loads(response.text)

In [58]:
data.keys()

dict_keys(['elementList', 'total', 'totalPages', 'actualPage', 'itemsPerPage', 'numPaginations', 'hiddenResults', 'summary', 'filter', 'alertName', 'lowerRangePosition', 'upperRangePosition', 'paginable'])

In [59]:
len(data['elementList'])

20

In [60]:
data['totalPages']

885

In [62]:
data['elementList'][0].keys()

dict_keys(['propertyCode', 'externalReference', 'numPhotos', 'floor', 'price', 'propertyType', 'operation', 'size', 'exterior', 'rooms', 'bathrooms', 'address', 'province', 'municipality', 'district', 'country', 'neighborhood', 'latitude', 'longitude', 'showAddress', 'url', 'description', 'hasVideo', 'status', 'newDevelopment', 'hasLift', 'parkingSpace', 'priceByArea', 'detailedType', 'suggestedTexts', 'hasPlan', 'has3DTour', 'has360', 'hasStaging', 'superTopHighlight', 'topNewDevelopment'])

In [63]:
data['elementList'][0]['priceByArea']

16949.0

In [66]:
data['elementList'][0]['priceByArea'] == int(data['elementList'][0]['price']/data['elementList'][0]['size'])

True

In [68]:
range(data['totalPages'])

range(0, 885)